<a href="https://colab.research.google.com/github/KevinTheRainmaker/MovieRecSys/blob/main/MovieRecSys.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/data/TMDB_5000/' 

Mounted at /content/gdrive


## Dataset

TMDB 5000 Datasets from Kaggle

In [2]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

movies = pd.read_csv(os.path.join(root_path, 'tmdb_5000_movies.csv'))
credits = pd.read_csv(os.path.join(root_path, 'tmdb_5000_credits.csv'))

In [5]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [6]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


### Combine datasets

In [7]:
credits.rename(columns = {'movie_id' : 'id'}, inplace = True)

In [8]:
df = movies.merge(credits, on=['title', 'id'])
df.sample()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
3753,4000000,"[{""id"": 18, ""name"": ""Drama""}]",NaN,85350,"[{""id"": 1850, ""name"": ""daily life""}, {""id"": 27...",en,Boyhood,The film tells a story of a divorced couple tr...,43.40352,"[{""name"": ""IFC Films"", ""id"": 307}, {""name"": ""D...",...,44349000,164.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}, ...",Released,12 years in the making.,Boyhood,7.5,1971,"[{""cast_id"": 3, ""character"": ""Mason"", ""credit_...","[{""credit_id"": ""52fe49369251416c910a6413"", ""de..."


### Simple EDA

In [9]:
df.shape

(4803, 22)

In [10]:
df['original_language'].value_counts()

en    4505
fr      70
es      32
zh      27
de      27
hi      19
ja      16
it      14
cn      12
ru      11
ko      11
pt       9
da       7
sv       5
nl       4
fa       4
th       3
he       3
ta       2
cs       2
ro       2
id       2
ar       2
vi       1
sl       1
ps       1
no       1
ky       1
hu       1
pl       1
af       1
nb       1
tr       1
is       1
xx       1
te       1
el       1
Name: original_language, dtype: int64

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [12]:
df.isnull().sum()

budget                     0
genres                     0
homepage                3091
id                         0
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64

In [14]:
df.duplicated().sum()

0

### Extract some features
- id
- title
- overview
- genres
- keywords
- cast
- crew
- popularity
- vote average
- vote count

In [135]:
extract = df[['id','title','overview','genres','keywords','cast','crew', 'popularity','vote_average','vote_count']].copy()

# drop missing values (overview)
extract.dropna(inplace=True)

In [136]:
pd.set_option('max_colwidth', 100)
extract[['genres','keywords']].sample()

,genres,keywords
2692,"[{""id"": 27, ""name"": ""Horror""}, {""id"": 53, ""name"": ""Thriller""}]","[{""id"": 3405, ""name"": ""blow job""}, {""id"": 10541, ""name"": ""curse""}, {""id"": 156092, ""name"": ""fat s..."


In [137]:
extract.shape

(4800, 10)

In [138]:
extract.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 0 to 4802
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4800 non-null   int64  
 1   title         4800 non-null   object 
 2   overview      4800 non-null   object 
 3   genres        4800 non-null   object 
 4   keywords      4800 non-null   object 
 5   cast          4800 non-null   object 
 6   crew          4800 non-null   object 
 7   popularity    4800 non-null   float64
 8   vote_average  4800 non-null   float64
 9   vote_count    4800 non-null   int64  
dtypes: float64(2), int64(2), object(6)
memory usage: 412.5+ KB


### Data Preprocessing

In [139]:
type(extract['genres'][0])

str

In [140]:
# genres, keywords: str to list 
from ast import literal_eval
extract['genres'] = extract['genres'].apply(literal_eval)
extract['keywords'] = extract['keywords'].apply(literal_eval)

In [141]:
type(extract['genres'][0])

list

In [142]:
extract['genres'][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

In [143]:
# extract value only from 'name' key in dictionary
extract['genres'] = extract['genres'].apply(lambda x : [y['name'] for y in x])
extract['keywords'] = extract['keywords'].apply(lambda x : [y['name'] for y in x])
extract['keywords'] = extract['keywords'].apply(lambda x:[i.replace(" ", "-") for i in x])

In [144]:
extract['genres'][0]

['Action', 'Adventure', 'Fantasy', 'Science Fiction']

In [145]:
extract['keywords'][0]

['culture-clash',
 'future',
 'space-war',
 'space-colony',
 'society',
 'space-travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien-planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love-affair',
 'anti-war',
 'power-relations',
 'mind-and-soul',
 '3d']

In [146]:
type(extract['genres'][0])

list

In [147]:
# extract['genres'] = extract['genres'].apply(lambda x:[i.lower().replace(" ", "_") for i in x]) # ex: Science Fiction to science_fiction
# extract['keywords'] = extract['keywords'].apply(lambda x:[i.replace(" ", "_") for i in x]) # ex: space war to space_war

In [148]:
extract[['genres','keywords']].sample()

,genres,keywords
1108,"[Animation, Family]","[italy, lie, magic, fairy, pinocchio, carnival, wish, boy, nose, puppet, animation, pool, consci..."


In [149]:
# cast
pd.set_option('max_colwidth', 1000)
extract[['cast']].head(1)

,cast
0,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""credit_id"": ""5602a8a7c3a3685532001c9a"", ""gender"": 2, ""id"": 65731, ""name"": ""Sam Worthington"", ""order"": 0}, {""cast_id"": 3, ""character"": ""Neytiri"", ""credit_id"": ""52fe48009251416c750ac9cb"", ""gender"": 1, ""id"": 8691, ""name"": ""Zoe Saldana"", ""order"": 1}, {""cast_id"": 25, ""character"": ""Dr. Grace Augustine"", ""credit_id"": ""52fe48009251416c750aca39"", ""gender"": 1, ""id"": 10205, ""name"": ""Sigourney Weaver"", ""order"": 2}, {""cast_id"": 4, ""character"": ""Col. Quaritch"", ""credit_id"": ""52fe48009251416c750ac9cf"", ""gender"": 2, ""id"": 32747, ""name"": ""Stephen Lang"", ""order"": 3}, {""cast_id"": 5, ""character"": ""Trudy Chacon"", ""credit_id"": ""52fe48009251416c750ac9d3"", ""gender"": 1, ""id"": 17647, ""name"": ""Michelle Rodriguez"", ""order"": 4}, {""cast_id"": 8, ""character"": ""Selfridge"", ""credit_id"": ""52fe48009251416c750ac9e1"", ""gender"": 2, ""id"": 1771, ""name"": ""Giovanni Ribisi"", ""order"": 5}, {""cast_id"": 7, ""character"": ""Norm Spellman"", ""credit_id"": ""52fe48009251416c750ac9dd"", ""gender..."


In [150]:
def convert_cast(obj):
  L = []
  counter = 0
  for i in literal_eval(obj):
    if counter != 3: # max: 3
      L.append(i['name'])
      counter+=1
    else:
      break
  return L

In [151]:
extract['cast'] = extract['cast'].apply(convert_cast)
extract['cast'] = extract['cast'].apply(lambda x:[i.replace(" ", "") for i in x])

In [152]:
extract[['cast']].sample()

,cast
3087,"[CharlieHunnam, NathanLane, JimBroadbent]"


In [153]:
# crew
def fetch_director(obj):
  L = []
  for i in literal_eval(obj):
    if i['job']=='Director':
      L.append(i['name'])
      break
  return L

In [154]:
# crew to director
extract['crew'] = extract['crew'].apply(fetch_director)
extract.rename(columns = {'crew' : 'director'}, inplace = True)
extract['director'] = extract['director'].apply(lambda x:[i.replace(" ", "") for i in x])

In [155]:
extract[['director']].head(1)

,director
0,[JamesCameron]


In [156]:
# overview
# extract['overview'] = extract['overview'].apply(lambda x: x.lower().split())

In [157]:
# extract.sample()

In [158]:
extract.columns

Index(['id', 'title', 'overview', 'genres', 'keywords', 'cast', 'director',
       'popularity', 'vote_average', 'vote_count'],
      dtype='object')

In [159]:
extract['tags'] = extract['genres'] + extract['cast'] + extract['director']
movies_df = extract#[['id','title','overview','tags']]

In [160]:
movies_df.sample()

,id,title,overview,genres,keywords,cast,director,popularity,vote_average,vote_count,tags
417,16281,Creepshow,"Inspired by the E.C. comics of the 1950s, George A.Romero and Stephen King bring five tales of terror to the screen.","[Horror, Comedy, Fantasy]","[monster, halloween, meteor, buried-alive, cockroach, anthology, based-on-comic-book, gore, animated-sequence, zombie, living-dead]","[HalHolbrook, AdrienneBarbeau, FritzWeaver]",[GeorgeA.Romero],13.661289,6.7,226,"[Horror, Comedy, Fantasy, HalHolbrook, AdrienneBarbeau, FritzWeaver, GeorgeA.Romero]"


In [161]:
movies_df.shape

(4800, 11)

In [162]:
movies_df['keywords'][0]

['culture-clash',
 'future',
 'space-war',
 'space-colony',
 'society',
 'space-travel',
 'futuristic',
 'romance',
 'space',
 'alien',
 'tribe',
 'alien-planet',
 'cgi',
 'marine',
 'soldier',
 'battle',
 'love-affair',
 'anti-war',
 'power-relations',
 'mind-and-soul',
 '3d']

## Measure Content Simillarity

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# genres
movies_df['genres_literal'] = movies_df['genres'].apply(lambda x: (' ').join(x)) 

count_vect = CountVectorizer(min_df=0, ngram_range=(1,2)) 
# max_df / min_df: 토큰이 나타난 횟수를 기준으로, max_df 값보다 크거나, min_df 값보다 작으면 무시
# ngram_range: (min_n, max_n)으로, BoW 생성에 사용할 토큰의 크기인 n-gram의 범위를 결정 - 여기서는 최소 모노그램, 최대 바이그램

genre_mat = count_vect.fit_transform(movies_df['genres_literal']) # csr_matrix: CSR 형식 희소 행렬
genre_mat.shape

(4800, 272)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim)

(4800, 4800)
[[1.         0.6761234  0.50709255 ... 0.         0.         0.        ]
 [0.6761234  1.         0.4        ... 0.         0.         0.        ]
 [0.50709255 0.4        1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [ ]:
genre_sim_sorted_ind = genre_sim.argsort()[:, ::-1]
print(genre_sim_sorted_ind[:1])

[[   0   14 3493 ... 3037 3036 2399]]


In [ ]:
# keywords
movies_df['keywords_literal'] = movies_df['keywords'].apply(lambda x: (' ').join(x)) 

count_vect = CountVectorizer(min_df=0, ngram_range=(1,10)) 

keyword_mat = count_vect.fit_transform(movies_df['keywords_literal']) # csr_matrix: CSR 형식 희소 행렬
keyword_mat.shape

(4800, 178859)

In [ ]:
keyword_sim = cosine_similarity(keyword_mat, keyword_mat)
print(keyword_sim.shape)
print(keyword_sim)

(4800, 4800)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


In [ ]:
keyword_sim_sorted_ind = keyword_sim.argsort()[:, ::-1]
print(keyword_sim_sorted_ind[:1])

[[   0  492   47 ... 3176 3177 2399]]


In [ ]:
# cast + director = people
movies_df['people'] = movies_df['cast'] + movies_df['director']
movies_df['people_literal'] = movies_df['people'].apply(lambda x: (' ').join(x)) 
count_vect = CountVectorizer(min_df=0, ngram_range=(1,3)) 
people_mat = count_vect.fit_transform(movies_df['people_literal'])
people_sim = cosine_similarity(people_mat, people_mat)
people_sim_sorted_ind = people_sim.argsort()[:, ::-1]

In [ ]:
print(people_sim_sorted_ind[:1])

[[   0 2403 1245 ... 3202 3203 2399]]


## Recommendation using Contents Filtering

In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    #top_n의 2배에 해당하는 유사 장르 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [ ]:
c = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(0.6)
print('c:', round(c,3), '\nm:', round(m,3))

c: 6.093 
m: 371.0


In [ ]:
percentile = 0.6

c = movies_df['vote_average'].mean()
m = movies_df['vote_count'].quantile(percentile)

def weighted_vote_average(record):
    v = record['vote_count']
    r = record['vote_average']
#     print(v,r)
    return ((v/(v+m)) * r) + ((m/(v+m)) * c)

movies_df['weighted_vote'] = movies_df.apply(weighted_vote_average, axis=1)

In [ ]:
movies_df[['title','vote_average','weighted_vote', 'vote_count']].sort_values('weighted_vote', ascending=False)[:10]

,title,vote_average,weighted_vote,vote_count
1881,The Shawshank Redemption,8.5,8.395869,8205
3337,The Godfather,8.4,8.263358,5893
662,Fight Club,8.3,8.216309,9413
3232,Pulp Fiction,8.3,8.206941,8428
65,The Dark Knight,8.2,8.136820,12002
1818,Schindler's List,8.3,8.125781,4329
3865,Whiplash,8.3,8.122956,4254
809,Forrest Gump,8.2,8.105793,7927
2294,Spirited Away,8.3,8.105550,3840
2731,The Godfather: Part II,8.3,8.079232,3338


In [ ]:
def find_sim_movie(df, sorted_ind, title_name, top_n = 10):
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    #top_n의 2배에 해당하는 유사 장르 인덱스 추출
    similar_indexes = sorted_ind[title_index, :(top_n * 2)]
    similar_indexes = similar_indexes.reshape(-1)
    
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes != title_index]
    
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:top_n]

In [ ]:
similar_movies = find_sim_movie(movies_df, genre_sim_sorted_ind, 'The Godfather', 10)
similar_movies[['title','vote_average','weighted_vote']]

,title,vote_average,weighted_vote
875,Moulin Rouge!,7.4,7.109798
637,Les Misérables,7.1,6.934311
1684,Walk the Line,7.3,6.888771
4704,Once,7.3,6.756520
1080,Across the Universe,7.1,6.621603
3211,Crazy Heart,6.8,6.393290
2524,Coal Miner's Daughter,7.2,6.262265
2350,Center Stage,6.8,6.230959
4097,We Are Your Friends,6.3,6.225337
3212,The Rose,6.8,6.142537


In [ ]:
def recommend(movie):
  movie_index = movies_df[movies_df['title'] == movie].index[0]
  distances = similar_movies[movie]

,id,title,overview,genres,keywords,cast,popularity,vote_average,vote_count,director,tags,genres_literal,keywords_literal,people,people_literal,weighted_vote
875,824,Moulin Rouge!,A celebration of love and creative inspiration...,"[drama, music, romance]","[duke, musical, writer's_block, music, termina...","[NicoleKidman, EwanMcGregor, JohnLeguizamo]",57.374341,7.4,1300,[BazLuhrmann],"[drama, music, romance, duke, musical, writer'...",drama music romance,duke musical writer's_block music terminal_ill...,"[NicoleKidman, EwanMcGregor, JohnLeguizamo, Ba...",NicoleKidman EwanMcGregor JohnLeguizamo BazLuh...,7.109798
637,82695,Les Misérables,An adaptation of the successful stage musical ...,"[drama, music, romance]","[france, robbery, brothel, mayor, star, musica...","[HughJackman, RussellCrowe, AnneHathaway]",48.356214,7.1,1884,[TomHooper],"[drama, music, romance, france, robbery, broth...",drama music romance,france robbery brothel mayor star musical arre...,"[HughJackman, RussellCrowe, AnneHathaway, TomH...",HughJackman RussellCrowe AnneHathaway TomHooper,6.934311
1684,69,Walk the Line,A chronicle of country music legend Johnny Cas...,"[drama, music, romance]","[germany, prison, music_record, adultery, coun...","[JoaquinPhoenix, ReeseWitherspoon, GinniferGoo...",35.580032,7.3,718,[JamesMangold],"[drama, music, romance, germany, prison, music...",drama music romance,germany prison music_record adultery country_m...,"[JoaquinPhoenix, ReeseWitherspoon, GinniferGoo...",JoaquinPhoenix ReeseWitherspoon GinniferGoodwi...,6.888771
4704,5723,Once,A vacuum repairman moonlights as a street musi...,"[drama, music, romance]","[rock_and_roll, pop, irland, music_style, love...","[GlenHansard, MarkétaIrglová, HughWalsh]",19.052179,7.3,453,[JohnCarney],"[drama, music, romance, rock_and_roll, pop, ir...",drama music romance,rock_and_roll pop irland music_style love_of_o...,"[GlenHansard, MarkétaIrglová, HughWalsh, JohnC...",GlenHansard MarkétaIrglová HughWalsh JohnCarney,6.756520
1080,4688,Across the Universe,Musical based on The Beatles songbook and set ...,"[adventure, drama, music, romance]","[riot, protest, musical, music, cultural_diffe...","[EvanRachelWood, JimSturgess, JoeAnderson]",11.872841,7.1,410,[JulieTaymor],"[adventure, drama, music, romance, riot, prote...",adventure drama music romance,riot protest musical music cultural_difference...,"[EvanRachelWood, JimSturgess, JoeAnderson, Jul...",EvanRachelWood JimSturgess JoeAnderson JulieTa...,6.621603
3211,25196,Crazy Heart,When reporter Jean Craddock interviews Bad Bla...,"[drama, music, romance]","[taxi, country_music, journalist, guitar, bar,...","[JeffBridges, MaggieGyllenhaal, ColinFarrell]",17.697042,6.8,274,[ScottCooper],"[drama, music, romance, taxi, country_music, j...",drama music romance,taxi country_music journalist guitar bar music...,"[JeffBridges, MaggieGyllenhaal, ColinFarrell, ...",JeffBridges MaggieGyllenhaal ColinFarrell Scot...,6.393290
2524,16769,Coal Miner's Daughter,"Biography of Loretta Lynn, a country and weste...","[drama, music]","[country_music, female_friendship, biography, ...","[SissySpacek, TommyLeeJones, LevonHelm]",3.826685,7.2,67,[MichaelApted],"[drama, music, country_music, female_friendshi...",drama music,country_music female_friendship biography lore...,"[SissySpacek, TommyLeeJones, LevonHelm, Michae...",SissySpacek TommyLeeJones LevonHelm MichaelApted,6.262265
2350,10560,Center Stage,A group of 12 teenagers from various backgroun...,"[drama, music]","[new_york, competition, dancer, dance, ball, c...","[AmandaSchull, ZoeSaldana, SusanMayPratt]",6.961736,6.8,90,[NicholasHytner],"[drama, music, new_york, competition, dancer, ...",drama music,new_york competition dancer dance ball career ...,"[AmandaSchull, ZoeSaldana, SusanMayPratt, Nich...",AmandaSchull ZoeSaldana SusanMayPratt Nicholas...,6.230959
4097,301351,We Are Your Friends,Young Cole Carter dreams of hitting the big ti...,"[drama, music, romance]",[dj],"[ZacEfron, EmilyRatajkowski, WesBentley]",2